# Mount Drive



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries and initializing stopwords and stemmer


In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import nltk

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
from sklearn.preprocessing import LabelEncoder

# Reading Data


In [86]:
hasoc_train=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/en_Hasoc2021_train.csv") # hasoc only data
# combined data of hasoc+kaggle
hasoc_test=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/en_Hasoc2021_test_data.csv") # hasoc test data
hasoc_test_bin=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/1A_English_actual_labels.csv") # hasoc binary test labels
hasoc_test_mul=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/1B_English_actual_labels.csv") #hasoc multiclass test labels

In [60]:
combined_train=pd.read_csv("/content/drive/MyDrive/ML JCOMP SEM5/hate_bin_combine.csv",encoding = "ISO-8859-1") 

In [ ]:
hasoc_train

,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN
...,...,...,...,...,...
3838,1661,60c5d6bf5659ea5e55defd57,@BBCNews Let the dog deal with the wanker once...,HOF,PRFN
3839,194,60c5d6bf5659ea5e55def185,India has suffered a lot. That Chinese bastard...,HOF,HATE
3840,3988,60c5d6bf5659ea5e55def78c,People didn't give 300+ seats majority to BJP ...,HOF,HATE
3841,4212,60c5d6bf5659ea5e55defb04,"@KanganaTeam This is such a vile, xenophobic a...",HOF,PRFN


In [ ]:
combined_train

,_id,text,task_1
0,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF
1,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF
2,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT
3,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF
4,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF
...,...,...,...
35800,31958,ate @user isz that youuu?ÃÂ°ÃÂÃÂÃÂÃÂ°Ã...,NOT
35801,31959,to see nina turner on the airwaves trying to w...,NOT
35802,31960,listening to sad songs on a monday morning otw...,NOT
35803,31961,"@user #sikh #temple vandalised in in #calgary,...",HOF


In [87]:
hasoc_test["task_1"]=hasoc_test_bin["label"]

In [88]:
hasoc_test["task_2"]=hasoc_test_mul["label"]

# Basic Preprocessing of Tweets


In [63]:
lemmatizer = WordNetLemmatizer()
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not"])


In [64]:
def preprocess(tweets):
  
  tweets.encode('ascii', 'ignore').decode('ascii') #remove emojis
  tweets=tweets.lower()                   #convert to lower case
  tweets = re.sub(r"http\S+", " ", tweets) #remove urls
  l=tweets.split(" ")
  for n,i in enumerate(l):
    if '@' in i:
      l[n]=""
  
  tweets=" ".join(l)
  tweets= re.sub('[^A-Za-z]+',' ',tweets) #remove bad character
  tweets = [word for word in tweets.split(" ") if not word in stop_words] # removing stop words
  tweets= [lemmatizer.lemmatize(token, "v") for token in tweets] #Lemmatization
  tweets=" ".join(tweets) 
  return tweets 
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text


## when using only hasoc

In [89]:
hasoc_train["text"]=hasoc_train["text"].apply(lambda x:decontract(x))
hasoc_train["text"]=hasoc_train["text"].apply(lambda x:preprocess(x))

In [90]:
train_df, val_df = train_test_split(hasoc_train, test_size = 0.2, random_state = 42)
train_tweet=train_df.text
val_tweet=val_df.text

## when using combined data

In [65]:
combined_train["text"]=combined_train["text"].apply(lambda x:decontract(x))
combined_train["text"]=combined_train["text"].apply(lambda x:preprocess(x))

In [66]:
train_df, val_df = train_test_split(combined_train, test_size = 0.2, random_state = 42)
train_tweet=train_df.text
val_tweet=val_df.text

## preprocessing the testdata

In [91]:
hasoc_test["text"]=hasoc_test["text"].apply(lambda x:decontract(x))
hasoc_test["text"]=hasoc_test["text"].apply(lambda x:preprocess(x))

In [92]:
test_tweet=hasoc_test.text

In [93]:
le=LabelEncoder()

## on evaluation data

for binary data

In [94]:
train_df["task_1"] = le.fit_transform(train_df["task_1"])
val_df["task_1"] = le.fit_transform(val_df["task_1"])

In [95]:
hasoc_test["task_1"]=le.fit_transform(hasoc_test["task_1"])

for multiclass data

In [ ]:
train_df["task_2"] = le.fit_transform(train_df["task_2"])
val_df["task_2"] = le.fit_transform(val_df["task_2"])

In [ ]:
hasoc_test["task_2"]=le.fit_transform(hasoc_test["task_2"])

## run for both

In [96]:
full_train_tweet=hasoc_train["text"]

In [97]:
full_train_tweet=combined_train["text"] # run this only for combined data

In [98]:
train_tweet=train_df["text"]

In [99]:
val_tweet=val_df["text"]

### FOR BINARY

In [100]:
t1=train_df["task_1"]
t2=val_df["task_1"]

In [101]:
t3=hasoc_test["task_1"]

## FOR MULTICLASS

In [ ]:
t4=train_df["task_2"]
t5=val_df["task_2"]
t6=hasoc_test["task_2"]

In [26]:
from sklearn.metrics import classification_report

#Basic LSTM model and training



In [102]:
top_words = 6000    #top 6000 words in the vocabulary
max_review_length = 130  #maximum sentence length
embedding_vector_length = 32  #each word is mapped to a 32 dimensional vector
tokenizer = Tokenizer(num_words=top_words) # top 6000 are getting tokenized
tokenizer.fit_on_texts(train_tweet)
list_tokenized_train = tokenizer.texts_to_sequences(train_tweet) # text data is converted to sequence data

## run this for binary only

In [103]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)  # making all input sequence into same length
y_train = t1

In [104]:
model = Sequential()  
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 130, 32)           192032    
                                                                 
 lstm_4 (LSTM)               (None, 100)               53200     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 245,333
Trainable params: 245,333
Non-trainable params: 0
_________________________________________________________________


### for binary classification

In [105]:
history = model.fit(X_train,y_train, epochs=30, batch_size=64)

Epoch 1/30
49/49 [==============================] - 9s 148ms/step - loss: 0.7730 - accuracy: 0.6295
Epoch 2/30
49/49 [==============================] - 7s 148ms/step - loss: 0.5313 - accuracy: 0.7115
Epoch 3/30
49/49 [==============================] - 7s 146ms/step - loss: 0.3840 - accuracy: 0.8679
Epoch 4/30
49/49 [==============================] - 7s 148ms/step - loss: 0.4747 - accuracy: 0.8774
Epoch 5/30
49/49 [==============================] - 7s 149ms/step - loss: 0.2045 - accuracy: 0.9258
Epoch 6/30
49/49 [==============================] - 7s 147ms/step - loss: 0.1471 - accuracy: 0.9486
Epoch 7/30
49/49 [==============================] - 7s 148ms/step - loss: 0.1217 - accuracy: 0.9655
Epoch 8/30
49/49 [==============================] - 7s 149ms/step - loss: 0.1048 - accuracy: 0.9736
Epoch 9/30
49/49 [==============================] - 7s 149ms/step - loss: 0.2009 - accuracy: 0.9525
Epoch 10/30
49/49 [==============================] - 7s 149ms/step - loss: 0.1220 - accuracy: 0.9750

###On Evaluation data

In [106]:
list_tokenized_test = tokenizer.texts_to_sequences(val_df['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = t2
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.7581274382314694
F1-score:  0.6172839506172839
Confusion matrix:


array([[433,  96],
       [ 90, 150]])

In [107]:
 print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.82      0.82       529
           1       0.61      0.62      0.62       240

    accuracy                           0.76       769
   macro avg       0.72      0.72      0.72       769
weighted avg       0.76      0.76      0.76       769



### on test data

In [108]:
list_tokenized_test = tokenizer.texts_to_sequences(hasoc_test['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = t3
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.7244340359094458
F1-score:  0.6175514626218851
Confusion matrix:


array([[643, 155],
       [198, 285]])

In [109]:
 print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.81      0.78       798
           1       0.65      0.59      0.62       483

    accuracy                           0.72      1281
   macro avg       0.71      0.70      0.70      1281
weighted avg       0.72      0.72      0.72      1281



## run this for Combined only

In [79]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = t1

In [80]:
model = Sequential()  
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 130, 32)           192032    
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 245,333
Trainable params: 245,333
Non-trainable params: 0
_________________________________________________________________


In [81]:
history = model.fit(X_train,y_train, epochs=30, batch_size=64)

Epoch 1/30
448/448 [==============================] - 70s 150ms/step - loss: 2.0482 - accuracy: 0.8495
Epoch 2/30
448/448 [==============================] - 67s 150ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 3/30
448/448 [==============================] - 68s 151ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 4/30
448/448 [==============================] - 68s 152ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 5/30
448/448 [==============================] - 68s 152ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 6/30
448/448 [==============================] - 68s 151ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 7/30
448/448 [==============================] - 67s 150ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 8/30
448/448 [==============================] - 67s 150ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 9/30
448/448 [==============================] - 68s 151ms/step - loss: 2.0219 - accuracy: 0.8674
Epoch 10/30
448/448 [==============================] - 67s 151ms/step - l

###On Evaluation data

In [82]:
list_tokenized_test = tokenizer.texts_to_sequences(val_df['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = t2
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.8680351906158358
F1-score:  0.9293563579277865
Confusion matrix:


array([[   0,  945],
       [   0, 6216]])

In [83]:
 print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       945
           1       0.87      1.00      0.93      6216

    accuracy                           0.87      7161
   macro avg       0.43      0.50      0.46      7161
weighted avg       0.75      0.87      0.81      7161



### on test data

In [84]:
list_tokenized_test = tokenizer.texts_to_sequences(hasoc_test['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = t3
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.3770491803278688
F1-score:  0.5476190476190476
Confusion matrix:


array([[  0, 798],
       [  0, 483]])

In [85]:
 print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       798
           1       0.38      1.00      0.55       483

    accuracy                           0.38      1281
   macro avg       0.19      0.50      0.27      1281
weighted avg       0.14      0.38      0.21      1281



## for multiclass

In [ ]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = t4

In [ ]:
model = Sequential()  
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           192032    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 245,333
Trainable params: 245,333
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train,y_train, epochs=30, batch_size=64)

Epoch 1/30
49/49 [==============================] - 10s 154ms/step - loss: nan - accuracy: 0.1763
Epoch 2/30
49/49 [==============================] - 7s 150ms/step - loss: nan - accuracy: 0.1763
Epoch 3/30
49/49 [==============================] - 7s 150ms/step - loss: nan - accuracy: 0.1763
Epoch 4/30
49/49 [==============================] - 7s 149ms/step - loss: nan - accuracy: 0.1763
Epoch 5/30
49/49 [==============================] - 7s 151ms/step - loss: nan - accuracy: 0.1763
Epoch 6/30
49/49 [==============================] - 7s 149ms/step - loss: nan - accuracy: 0.1763
Epoch 7/30
49/49 [==============================] - 7s 151ms/step - loss: nan - accuracy: 0.1763
Epoch 8/30
49/49 [==============================] - 7s 152ms/step - loss: nan - accuracy: 0.1763
Epoch 9/30
49/49 [==============================] - 7s 151ms/step - loss: nan - accuracy: 0.1763
Epoch 10/30
49/49 [==============================] - 7s 150ms/step - loss: nan - accuracy: 0.1763
Epoch 11/30
49/49 [=========

### on evaluation data

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(val_df['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = t5
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.18335500650195058
Confusion matrix:


array([[141,   0,   0,   0],
       [240,   0,   0,   0],
       [140,   0,   0,   0],
       [248,   0,   0,   0]])

### on test data

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(hasoc_test['text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = t6
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.17486338797814208
Confusion matrix:


array([[224,   0,   0,   0],
       [483,   0,   0,   0],
       [195,   0,   0,   0],
       [379,   0,   0,   0]])